In [ ]:
# 1. Install earthaccess
!pip install -q earthaccess
import os
from datetime import datetime, timedelta
import earthaccess


# Product-to-concept_id mapping
PRODUCTS = {
    "VNPIMG": "C3365190240-LPCLOUD",
    "VJ1IMG": "C3365180216-LPCLOUD",
    "VNPMOD": "C3365181544-LPCLOUD",
    "VJ1MOD": "C3365168551-LPCLOUD"
}

def ensure_drive():
    try:
        from google.colab import drive
        drive.mount('/content/drive')
    except ImportError:
        pass

def date_range(start_date, end_date):
    """Yield all dates between start_date and end_date (inclusive) as strings YYYY-MM-DD"""
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    for n in range((end - start).days + 1):
        yield (start + timedelta(days=n)).strftime("%Y-%m-%d")

def bulk_download(product, area, start_date, end_date, output_dir, provider="LP DAAC", threads=8):
    """
    Downloads Earthdata granules for a product, area, and date range.
    Args:
        product (str): 'VNPIMG', 'VJ1IMG', 'VNPMOD', or 'VJ1MOD'
        area (tuple): (west, south, east, north)
        start_date (str): 'YYYY-MM-DD'
        end_date (str): 'YYYY-MM-DD'
        output_dir (str): Save location
        provider (str): Default 'LP DAAC'
        threads (int): Number of parallel downloads
    """
    if product not in PRODUCTS:
        raise ValueError(f"Unknown product: {product}. Choose from: {list(PRODUCTS.keys())}")
    concept_id = PRODUCTS[product]

    os.makedirs(output_dir, exist_ok=True)
    earthaccess.login(strategy="interactive")

    total_urls = []
    for date in date_range(start_date, end_date):
        start_time = f"{date}T00:00:00Z"
        end_time = f"{date}T23:59:59Z"
        temporal = (start_time, end_time)
        print(f"\nSearching for {date} ...")
        results = earthaccess.search_data(
            concept_id=concept_id,
            temporal=temporal,
            bounding_box=area,
            cloud_hosted=True
        )
        print(f"Found {len(results)} granules for {date}")
        for granule in results:
            data_links = granule.data_links()
            if data_links:
                total_urls.append(data_links[0])

    print(f"\nReady to download {len(total_urls)} granules to {output_dir} ...")
    earthaccess.download(total_urls, local_path=output_dir, provider=provider, threads=threads)
    print("Download completed.")



# Example usage (uncomment and set your parameters for Colab or script use)
if __name__ == "__main__" or True:  # Force run in Colab cell
    ensure_drive()
    # Parameters
    # Choose one: 'VNPIMG', 'VJ1IMG', 'VNPMOD', 'VJ1MOD'
    product = "VNPIMG"
    concept_id = "C3365190240-LPCLOUD"  # VIIRS VNP47IMG.002
    area = (-121.8, 39.5, -121.3, 40)  # (west, south, east, north)
    start_date = "2018-11-09"
    end_date = "2018-11-20"
    output_dir = "/content/drive/MyDrive/FILDA_DATA"

    bulk_download(
        product=product,
        area=area,
        start_date=start_date,
        end_date=end_date,
        output_dir=output_dir,
        provider="LP DAAC",
        threads=4,
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Searching for 2018-11-09 ...
Found 1 granules for 2018-11-09

Searching for 2018-11-10 ...
